In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import json

In [ ]:
raw_data = pd.read_csv('detailed-data.csv')
# To see all the (unique) devices + data streams:
# raw_data.drop(columns=['from', 'to', 'value']).drop_duplicates()
# device_identifiers = {
#     "Huis": {
#         "Ventilatie_snelheid": ("uuid_a", "FanSpeedPercentage", "FanSpeed"),
#         "Ventilatie_ppm_co2": ("uuid_b", "PartsPerMillion", "CO2"),
#         "Gas_gebruik": ("uuid_c", "CentralMeterGasUsage", "m3"),
#         "Electriciteit_gebruik_hoog": ("uuid_d", "ElectricityUsage", "Wh"),
#         "Electriciteit_gebruik_laag": ("uuid_e", "ElectricityUsage", "Wh"),
#         "Electriciteit_teruglevering_hoog": ("uuid_f", "ElectricityProduction", "Wh"),
#         "Electriciteit_teruglevering_laag": ("uuid_g", "ElectricityProduction", "Wh"),
#     },
#     "Bar": {
#         "Temperatuur_gemeten": ("uuid_a", "ActualTemperatureCelcius", "c"),
#         "Temperatuur_ingesteld": ("uuid_b", "ConfiguredTemperatureCelcius", "c"),
#     },
# }
# kamers = ['Foo', 'Bar']

# _kamers = open("kamers", "w")
# _kamers.write(json.dumps(kamers))
# _kamers.close()

# _device_identifiers = open("device_identifiers", "w")
# _device_identifiers.write(json.dumps(device_identifiers))
# _device_identifiers.close()

In [ ]:
_kamers = open("kamers.json", "r")
kamers = json.loads(_kamers.read())
_kamers.close()

_device_identifiers = open("device_identifiers.json", "r")
device_identifiers = json.loads(_device_identifiers.read())
_device_identifiers.close()

In [ ]:
raw_data['_from'] = raw_data['from'].apply(lambda ts: pd.datetime.fromtimestamp(ts / 1000))
raw_data['_to'] = raw_data['to'].apply(lambda ts: pd.datetime.fromtimestamp(ts / 1000))

In [ ]:
devices = device_identifiers

for section, measurements in devices.items():
    for name, (uuid, c, u) in measurements.items():
        df = raw_data
        df = df[df['uuid'] == uuid]
        df = df[df['category'] == c]
        df = df[df['unit'] == u]
        df = df.set_index('_from').asfreq('15T')
        df = df.drop(columns=['from', 'to', 'uuid'])
        devices[section][name] = df

In [ ]:
sns.set(rc={'figure.figsize':(18, 6)})

fig, ax = plt.subplots()
for kamer in kamers:
    t = devices[kamer]['Temperatuur_gemeten']['value']
    ax.plot(t.index, t, marker='x', linestyle='-', linewidth=0.5, label=kamer)

t = devices['Woonkamer']['Temperatuur_ingesteld']['value']
ax.plot(t.index, t, marker='x', linestyle='-', linewidth=0.5, label='Ingesteld')

major = mdates.DayLocator(interval = 1)
ax.xaxis.set_major_locator(major)
ax.set_ylabel('Celcius')
ax.legend()

In [ ]:
kamers = ['Woonkamer', 'Badkamer', 'Slaapkamer_groot', 'Kantoor', 'Slaapkamer_dochter']
for kamer in kamers:
    fig, ax = plt.subplots()
    g = devices[kamer]['Temperatuur_gemeten']['value']
    i = devices[kamer]['Temperatuur_ingesteld']['value']
    ax.plot(g.index, g, marker='x', linestyle='-', linewidth=0.5, label='Gemeten')
    ax.plot(i.index, i, marker='x', linestyle='-', linewidth=0.5, label='Ingesteld')
    ax.set_ylabel(kamer + ' in celcius')
    ax.legend()


In [ ]:
fig, ax_fan = plt.subplots()
fan = devices['Huis']['Ventilatie_snelheid']['value']
co2 = devices['Huis']['Ventilatie_ppm_co2']['value']
ax_fan.plot(fan.index, fan, marker='x', linestyle='-', linewidth=0.5, label='Ventilatie snelheid')
ax_fan.set_ylabel('Ventilatie %')

ax_co2 = ax_fan.twinx()
ax_co2.plot(co2.index, co2, marker='o', linestyle='-', linewidth=0.5, label='PPM CO2')
ax_co2.set_ylabel('PPM CO2')
fig.legend()

In [ ]:
fig, ax_gas = plt.subplots()

gas = devices['Huis']['Gas_gebruik']['value']
ehoog_consumptie = devices['Huis']['Electriciteit_gebruik_hoog']['value']
elaag_consumptie = devices['Huis']['Electriciteit_gebruik_laag']['value']
ehoog_terug = devices['Huis']['Electriciteit_teruglevering_hoog']['value']
elaag_terug = devices['Huis']['Electriciteit_teruglevering_laag']['value']
ehuis = devices['Huis']['Electriciteit_opwek_dak']['value']
egarage = devices['Huis']['Electriciteit_opwek_garage']['value']

ax_gas.plot(gas.index, gas, marker='x', linestyle='-', linewidth=0.5, label='Gas')
ax_gas.set_ylabel('Gas m3')

ax_ehc = ax_gas.twinx()
ax_ehc.plot(ehoog_consumptie.index, ehoog_consumptie, marker='o', linestyle='-', linewidth=0.5, label='Consumptie hoog tarief Wh')
ax_ehc.set_ylabel('Consumptie hoog tarief Wh')

# ax_elc = ax_ehc.twinx()
# ax_elc.plot(elaag_consumptie.index, elaag_consumptie, marker='o', linestyle='-', linewidth=0.5, label='Consumptie laag tarief Wh')
# ax_elc.set_ylabel('Consumptie laag tarief Wh')


fig.legend()